In [1]:
# As usual, a bit of setup
import time, os, re, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rnn_layers import *
from solver import Solver
from rnn import RNN
from data import *
from configure import *
from utils import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [141]:
os.listdir(PATH)

FileNotFoundError: [Errno 2] No such file or directory: '~/Documents/Data/A320_300_20/'

In [110]:
usecols = COLUMNS
start, end = (260, 300)
flight_df = pd.read_csv(PATH + '2016-03-02-17-18-10_A320__ZJSY_6372_160302142102817.csv',
                        names=COLUMNS,
                        usecols=usecols,
                        skiprows=start,nrows=end - start
                       ).fillna(method='pad'
                       ).fillna(method='bfill')
arr_env = decompose_wind(flight_df)
arr_drv = compose_sstick(flight_df, sstick=False)

others      = [ACC_LNG, ACC_LAT, ACC_VRT, PITH, ROLL, *SINGLE_COL]
data_others = [abs_max(flight_df[COLUMNS[i]].values) for i in others]
arr_others  = np.r_[data_others].T

Xtrain = np.c_[arr_env, arr_drv, arr_others]

In [19]:
%%time
splited = split_data()

CPU times: user 1min 48s, sys: 9.33 s, total: 1min 58s
Wall time: 2min 14s


In [25]:
splited['f_train']

array(['2016-08-14-11-08-31_A319_B-1827_ZUCK_CQN2390_160814085450827.csv',
       '2016-09-16-10-40-03_A319_B-1652_ZSQD_CSN6358_160916075130652.csv',
       '2016-05-25-04-50-31_A319_B-1601_ZHCC_CSZ9329_160525023049601.csv',
       ...,
       '2016-10-21-01-00-02_A319_B-1802_ZLXY_CSN3173_161020225843802.csv',
       '2016-09-22-12-56-36_A319_B-1652_ZYCC_CSN6610_160922085342652.csv',
       '2016-11-07-07-53-10_A319_B-1651_ZBTJ_CSN6580_161107054203651.csv'],
      dtype='<U66')

In [57]:
print(*[COLUMNS[i] for i in [SS_CP,PITH_CP,ROLL_CP,SS_FO,PITH_FO,ROLL_FO]], sep='\n')

['_SSTICK_CAPT', '_SSTICK_CAPT-1', '_SSTICK_CAPT-2', '_SSTICK_CAPT-3']
['_PITCH_CAPT_SSTICK', '_PITCH_CAPT_SSTICK-1', '_PITCH_CAPT_SSTICK-2', '_PITCH_CAPT_SSTICK-3']
['_ROLL_CAPT_SSTICK', '_ROLL_CAPT_SSTICK-1', '_ROLL_CAPT_SSTICK-2', '_ROLL_CAPT_SSTICK-3']
['_SSTICK_FO', '_SSTICK_FO-1', '_SSTICK_FO-2', '_SSTICK_FO-3']
['_PITCH_FO_SSTICK', '_PITCH_FO_SSTICK-1', '_PITCH_FO_SSTICK-2', '_PITCH_FO_SSTICK-3']
['_ROLL_FO_SSTICK', '_ROLL_FO_SSTICK-1', '_ROLL_FO_SSTICK-2', '_ROLL_FO_SSTICK-3']


In [45]:
sscp = flight_df[COLUMNS[SS_CP]].values
ssfo = flight_df[COLUMNS[SS_FO]].values
p_cp = flight_df[COLUMNS[PITH_CP]].values
p_fo = flight_df[COLUMNS[PITH_FO]].values
r_cp = flight_df[COLUMNS[ROLL_CP]].values
r_fo = flight_df[COLUMNS[ROLL_FO]].values

a=sscp * p_cp + ssfo * p_fo
b=p_cp+p_fo

In [44]:
a[np.arange(40), abs(a).argmax(axis=1)]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.35156   ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.01073503,  0.92284501,  0.        , -1.31834996,  0.        ,
        0.        ,  0.        ,  0.92284501,  0.043945  ,  0.        ,
        1.09862494,  0.        ,  0.        ,  2.85642505,  0.        ,
       -2.94431496, -4.04294014, -4.04294014, -7.82220984, -9.00872517])

In [46]:
b[np.arange(40), abs(b).argmax(axis=1)]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.70311999,
       -0.35156   ,  0.        , -1.274405  , -0.57128501, -1.23046005,
       -1.09862494,  0.        ,  0.        ,  1.18651497,  1.40623999,
        1.01073503,  1.31834996, -1.36229503, -1.31834996, -0.43945   ,
        0.        ,  0.74706501,  0.92284501,  1.14257002,  1.09862494,
        1.31834996,  0.        ,  0.        ,  2.85642505, -0.043945  ,
       -2.94431496, -4.04294014, -4.04294014, -7.82220984, -9.00872517])

In [29]:
b

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.01073503,  1.31834996,  0.        , -1.23046005,  0.        ,
        0.        ,  0.        ,  0.92284501,  1.14257002,  0.        ,
        1.31834996,  0.        ,  0.        ,  2.85642505,  0.        ,
        0.        , -1.45018494,  0.52733999, -1.49413002, -0.26367   ])

In [31]:
?np.load

Signature: np.max(a, axis=None, out=None, keepdims=<class 'numpy._globals._NoValue'>)
Docstring:
Return the maximum of an array or maximum along an axis.

Parameters
----------
a : array_like
    Input data.
axis : None or int or tuple of ints, optional
    Axis or axes along which to operate.  By default, flattened input is
    used.

    .. versionadded:: 1.7.0

    If this is a tuple of ints, the maximum is selected over multiple axes,
    instead of a single axis or all the axes as before.
out : ndarray, optional
    Alternative output array in which to place the result.  Must
    be of the same shape and buffer length as the expected output.
    See `doc.ufuncs` (Section "Output arguments") for more details.

keepdims : bool, optional
    If this is set to True, the axes which are reduced are left
    in the result as dimensions with size one. With this option,
    the result will broadcast correctly against the input array.

    If the default value is passed, then `keepdims` wil